In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.discriminant_analysis import StandardScaler
from numpy import NaN

In [2]:
df = pd.read_csv("..\dataset\lc_14to16.csv")
print(df.shape)


C:\Users\dgoku\AppData\Local\Temp\ipykernel_31036\4060919579.py:1: DtypeWarning: Columns (19,59,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("..\dataset\lc_14to16.csv")


(1091131, 151)


In [3]:
header_df = pd.DataFrame(columns=df.columns)

# Save the header to a new CSV file
# Replace 'output_header.csv' with your desired output file name
header_df.to_csv('output_header.csv', index=False)

print("Header saved to output_header.csv")

Header saved to output_header.csv


In [4]:
relevant_features = [
    'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 
    'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 
    'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 
    'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec', 
    'revol_bal', 'revol_util', 'total_acc', 'collections_12_mths_ex_med', 
    'mths_since_last_major_derog', 'acc_now_delinq', 'tot_coll_amt', 
    'tot_cur_bal', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 
    'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 
    'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'num_bc_sats', 
    'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 
    'num_rev_tl_bal_gt_0', 'num_sats', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 
    'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', 
    'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit'
]

# Select only the relevant features
df = df[relevant_features]
non_numeric_features = df.select_dtypes(include='object').columns.tolist()
print(non_numeric_features)
print(df.info())
df.to_csv('relevant_features.csv', index=False)


['term', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1091131 entries, 0 to 1091130
Data columns (total 50 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   loan_amnt                    1091131 non-null  float64
 1   term                         1091131 non-null  object 
 2   int_rate                     1091131 non-null  float64
 3   installment                  1091131 non-null  float64
 4   grade                        1091131 non-null  object 
 5   sub_grade                    1091131 non-null  object 
 6   emp_length                   1027081 non-null  object 
 7   home_ownership               1091131 non-null  object 
 8   annual_inc                   1091131 non-null  float64
 9   verification_status          1091131 non-null  object 
 10  dti                          1091066 non-null  float64
 11  delinq_2yrs     

In [5]:
#convert term  to years. 
def extract_numeric_term(term):
    return int(re.search(r'\d+', term).group())

# Apply the function directly to df['term']
df['term'] = df['term'].apply(extract_numeric_term)

In [6]:
#convert grade to numeric
df['grade'] = df['grade'].apply(lambda x: ord(x) - ord('A'))



In [7]:
#convert sub_grade to numeric
def convert_sub_grade(sub_grade):
    return (ord(sub_grade[0]) - ord('A')) * 5 + (ord(sub_grade[1]) - ord('0'))

# Apply the function to the sub_grade column
df['sub_grade'] = df['sub_grade'].apply(convert_sub_grade)

In [8]:
#convert emp_len to numeric

# Function to convert emp_length to numeric

def convert_emp_length(emp_length):
    if pd.isna(emp_length):
        return np.nan  # Proper NaN value from NumPy
    elif emp_length == '< 1 year':
        return 0
    elif emp_length == '10+ years':
        return 11
    else:
        # Extract the numeric part and convert to integer
        return int(emp_length.split()[0])    

# Apply the function to the emp_length column
df['emp_length'] = df['emp_length'].apply(convert_emp_length)


In [9]:
categories = df['verification_status'].unique()

print("Categories in verification_status:", categories)


categories = df['home_ownership'].unique()

print("Categories in home_ownership:", categories)
encoder = OneHotEncoder(drop='first', sparse_output=False)  # drop='first' avoids multicollinearity

# Select columns to encode
columns_to_encode = ['verification_status', 'home_ownership']

# Fit and transform the selected columns
encoded_data = encoder.fit_transform(df[columns_to_encode])

# Convert the encoded data into a DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_encode))

# Step 3: Drop the original categorical columns and add the encoded columns
df.drop(columns_to_encode, axis=1, inplace=True)  # Drop original columns in-place
df = pd.concat([df, encoded_df], axis=1)  # Add the encoded columns back to the original DataFrame

# Final DataFrame after in-place One-Hot Encoding
print("\nDataFrame after One-Hot Encoding:")
print(df)

Categories in verification_status: ['Not Verified' 'Source Verified' 'Verified']
Categories in home_ownership: ['MORTGAGE' 'RENT' 'OWN' 'ANY']

DataFrame after One-Hot Encoding:
         loan_amnt  term  int_rate  installment  grade  sub_grade  emp_length  \
0           3600.0    36     13.99       123.03      2         14        11.0   
1          24700.0    36     11.99       820.28      2         11        11.0   
2          20000.0    60     10.78       432.66      1          9        11.0   
3          35000.0    60     14.85       829.90      2         15        11.0   
4          10400.0    60     22.45       289.91      5         26         3.0   
...            ...   ...       ...          ...    ...        ...         ...   
1091126    24000.0    60     12.79       543.50      2         11         7.0   
1091127    24000.0    60     10.49       515.74      1          8        11.0   
1091128    40000.0    60     10.49       859.56      1          8         9.0   
1091129    2

In [10]:



numeric_df = df.select_dtypes(include=[np.number])
scaler = StandardScaler()

# Step 2: Apply the scaler to standardize the numeric columns
# fit_transform() will compute the mean and std for scaling and return the standardized data
standardized_numeric_df = pd.DataFrame(scaler.fit_transform(numeric_df), columns=numeric_df.columns)

# Step 3: Replace the original numeric columns in df with the standardized ones
df[numeric_df.columns] = standardized_numeric_df


In [11]:

numeric_df = df.select_dtypes(include=[np.number])

# Calculate correlation matrix
corr_matrix = numeric_df.corr()

# Define a correlation threshold
correlation_threshold = 0.5

# Find features to drop
corr_matrix_abs = corr_matrix.abs()
upper = corr_matrix_abs.where(np.triu(np.ones(corr_matrix_abs.shape), k=1).astype(bool))

# Identify features to drop based on high correlation
features_to_drop = [column for column in upper.columns if any(upper[column] > correlation_threshold)]

# Create a report for retained and dropped features
retained_features = df.columns[~df.columns.isin(features_to_drop)]

print("Correlation Analysis Results:")
print("-----------------------------")

# Print strongly correlated feature pairs
print("Strongly correlated feature pairs:")
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > correlation_threshold:
            print(f"- {corr_matrix.columns[i]} and {corr_matrix.columns[j]}: {corr_matrix.iloc[i, j]:.2f}")

print("\nFeatures to be removed due to high correlation:")
for feature in features_to_drop:
    correlated_features = upper[upper[feature] > correlation_threshold].index.tolist()
    print(f"- {feature} (correlated with: {', '.join(correlated_features)})")

print("\nRetained features:")
print(", ".join(retained_features))
print(retained_features.shape)


df = df[retained_features]

Correlation Analysis Results:
-----------------------------
Strongly correlated feature pairs:
- loan_amnt and installment: 0.95
- int_rate and grade: 0.96
- int_rate and sub_grade: 0.99
- grade and sub_grade: 0.97
- delinq_2yrs and mths_since_last_delinq: -0.56
- fico_range_low and fico_range_high: 1.00
- fico_range_low and bc_open_to_buy: 0.51
- fico_range_high and bc_open_to_buy: 0.51
- mths_since_last_delinq and mths_since_last_major_derog: 0.69
- open_acc and total_acc: 0.71
- open_acc and num_bc_sats: 0.61
- open_acc and num_bc_tl: 0.52
- open_acc and num_op_rev_tl: 0.83
- open_acc and num_rev_accts: 0.65
- open_acc and num_rev_tl_bal_gt_0: 0.67
- open_acc and num_sats: 1.00
- pub_rec and pub_rec_bankruptcies: 0.61
- pub_rec and tax_liens: 0.72
- revol_bal and total_bal_ex_mort: 0.50
- revol_bal and total_bc_limit: 0.50
- revol_util and bc_util: 0.85
- revol_util and percent_bc_gt_75: 0.72
- total_acc and num_bc_tl: 0.62
- total_acc and num_il_tl: 0.67
- total_acc and num_op_rev_

df.info()

In [12]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1091131 entries, 0 to 1091130
Data columns (total 25 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   loan_amnt                            1091131 non-null  float64
 1   term                                 1091131 non-null  float64
 2   int_rate                             1091131 non-null  float64
 3   emp_length                           1027081 non-null  float64
 4   annual_inc                           1091131 non-null  float64
 5   dti                                  1091066 non-null  float64
 6   delinq_2yrs                          1091131 non-null  float64
 7   fico_range_low                       1091131 non-null  float64
 8   inq_last_6mths                       1091130 non-null  float64
 9   open_acc                             1091131 non-null  float64
 10  pub_rec                              1091131 non-null  float64
 11